In [1]:
import util
import config
import numpy as np
import active_mentions_filtering.active_mentions_filter as amf
import pandas as pd

In [2]:
from plotly import tools
from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [3]:
ctrl_ptsd = pd.concat([util.load_picke_file(config.CTRL_PTSD_DF), util.load_picke_file(config.CTRL_PTSD_HELD_OUT_DF)])

In [4]:
ctrl_depr = pd.concat([util.load_picke_file(config.CTRL_DEPR_DF), util.load_picke_file(config.CTRL_DEPR_HELD_OUT_DF)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [ ]:
valid_users = ctrl_ptsd.index.values
tweets, labels = amf.get_labelled_data_for_ptsd_mh(valid_users)
scores = amf.evaluate_pipeline(tweets, labels, amf.get_classifier_pipeline())

In [4]:
valid_users = ctrl_depr.index.values
tweets, labels = amf.get_labelled_data_for_depr_mh(valid_users)
scores = amf.evaluate_pipeline(tweets, labels, amf.get_classifier_pipeline())

fit_time 4.557(0.071), 
test_precision 0.832(0.059), 
test_recall 0.889(0.052), 
test_roc_auc 0.964(0.019), 
score_time 0.725(0.086), 
test_f1 0.858(0.035), 


In [5]:
ctrl_ptsd['ptsd_tweet_count'] = [(row['ptsd_probs'] > 0.5).sum() for index, row in ctrl_ptsd.iterrows()]
ctrl_ptsd['mental_health_count'] = [(row['mental_health_probs'] > 0.5).sum() for index, row in ctrl_ptsd.iterrows()]
ctrl_ptsd['mental_health_frac'] = [(row['mental_health_probs'] > 0.5).sum()/row['tweet_count'] for index, row in ctrl_ptsd.iterrows()]

In [6]:
ctrl_depr['depr_tweet_count'] = [(row['depr_probs'] > 0.5).sum() for index, row in ctrl_depr.iterrows()]
ctrl_depr['mental_health_count'] = [(row['mental_health_probs'] > 0.5).sum() for index, row in ctrl_depr.iterrows()]
ctrl_depr['mental_health_frac'] = [(row['mental_health_probs'] > 0.5).sum()/row['tweet_count'] for index, row in ctrl_depr.iterrows()]

In [38]:
ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_frac'].mean(), ctrl_ptsd['mental_health_count'].max()

(0.0078733454851189349, 63)

In [28]:
del ctrl_ptsd

In [32]:
ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_frac'].mean(), ctrl_depr['mental_health_count'].max()

(0.012214913521249296, 152)

In [ ]:
hist, bin_edges = np.histogram(ctrl_depr.loc[ctrl_depr['labels']==0]['mental_health_frac'], density=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
Z = ctrl_ptsd.loc[ctrl_ptsd['labels']==0]['mental_health_count'].values
X1 = np.sort(Z)
N = len(Z)
F1 = np.array(range(N))/float(N)

Z = ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_count'].values
X2 = np.sort(Z)
N = len(Z)
F2 = np.array(range(N))/float(N)

In [ ]:
plt.plot(X1, F1)
plt.plot(X2, F2)

In [ ]:
Z = ctrl_depr.loc[ctrl_depr['labels']==0]['mental_health_count'].values
X1 = np.sort(Z)
N = len(Z)
F1 = np.array(range(N))/float(N)

Z = ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_count'].values
X2 = np.sort(Z)
N = len(Z)
F2 = np.array(range(N))/float(N)

plt.plot(X1, F1)
plt.plot(X2, F2)
# plt.show()

In [15]:
ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_frac'].mean()

0.012214913521249296

In [18]:
ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_frac'].mean()

0.013209038258285068

In [7]:
trace1 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==0]['mental_health_frac'],
    name='Control',
    histnorm='probability',
    opacity=0.5,
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)
trace2 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_frac'],
    name='Depression',
    histnorm='probability',
    opacity=0.5,
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='overlay', 
#     title='Depression vs. Control',
    xaxis=dict(title='Fraction of tweets removed'),
    yaxis=dict(title='Fraction of users'),
    width=600,
    height=400,
    legend=dict(orientation="h",x=0.5, y=0.96)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [7]:
trace1 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==0]['mental_health_count'],
    name='Control',
    opacity=0.5,
    xbins=dict(
        start=0,
        end=200,
        size=1
    )
)
trace2 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_count'],
    name='Depression',
    opacity=0.5,
        xbins=dict(
        start=0,
        end=200,
        size=1
    )
)

data = [trace1, trace2]
layout = go.Layout(barmode='overlay', title='Number of mental health related Tweets removed')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [9]:
trace1 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==0]['mental_health_frac'],
    name='Control',
    opacity=0.5,
    histnorm='probability',
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)
trace2 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_frac'],
    name='PTSD',
    opacity=0.5,
    histnorm='probability',
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='overlay', 
#     title='Depression vs. Control',
    xaxis=dict(title='Fraction of tweets removed'),
    yaxis=dict(title='Fraction of users'),
    width=600,
    height=400,
    legend=dict(orientation="h",x=0.6, y=0.95)
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [9]:
trace1 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==0]['mental_health_count'],
    name='Control',
    opacity=0.5,
    xbins=dict(
        start=0,
        end=200,
        size=1
    )
)
trace2 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_count'],
    name='PTSD',
    opacity=0.5,
        xbins=dict(
        start=0,
        end=200,
        size=1
    )
)

data = [trace1, trace2]
layout = go.Layout(barmode='overlay', title='Number of mental health related Tweets removed')
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [7]:
trace1 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==0]['mental_health_frac'],
    name='Control',
    opacity=0.5,
    histnorm='probability',
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)
trace2 = go.Histogram(
    x=ctrl_depr.loc[ctrl_depr['labels']==1]['mental_health_frac'],
    name='Depr',
    opacity=0.5,
    histnorm='probability',
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)


In [8]:
trace3 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==0]['mental_health_frac'],
    name='Control',
    opacity=0.5,
    histnorm='probability',
    marker=dict(color='#1f77b4'),
    showlegend=False,
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)
trace4 = go.Histogram(
    x=ctrl_ptsd.loc[ctrl_ptsd['labels']==1]['mental_health_frac'],
    name='PTSD',
    opacity=0.5,
    histnorm='probability',
    xbins=dict(
        start=0.0,
        end=0.05,
        size=0.002
    )
)

In [18]:

fig = tools.make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('DvC', 'PvC'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)


fig['layout'].update(height=330, width=700, barmode='overlay', font=dict(size=17), title='Fraction of Tweets Removed',  yaxis=dict(title='Fraction of users'), legend=dict(orientation='h', x=0.3, y=-0.15))

# fig['layout']['xaxis1'].update(title='xaxis 1 title')
# fig['layout']['xaxis2'].update(title='xaxis 2 title')
# fig['layout']['xaxis3'].update(title='xaxis 3 title')
# fig['layout']['xaxis4'].update(title='xaxis 4 title')
py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



Depression tweets in control class 
======

In [ ]:
for i, r in ctrl_depr.loc[ctrl_depr['labels']==0].iterrows():
    print('\n'.join(r['tweets'][r['mental_health_probs'] > 0.3]))

In [ ]:
ctrl_depr.loc[ctrl_depr['labels']==0].sort_values(['mental_health_count'], ascending=False)['mental_health_count']

In [ ]:
ctrl_depr.loc[ctrl_depr['labels']==0].sort_values(['depr_tweet_count'], ascending=False)['depr_tweet_count']

In [ ]:
cumsum = np.cumsum(ctrl_ptsd.loc[ctrl_ptsd['labels']==0]['depr_tweet_perc'])

trace1 = go.Scatter(x=[i for i in range(len(cumsum))], y=10*cumsum/np.linalg.norm(cumsum),
                     marker=dict(color='rgb(150, 25, 120)'))

cumsum = np.cumsum(ctrl_depr.loc[ctrl_depr['labels']==1]['depr_tweet_perc'])

trace2 = go.Scatter(x=[i for i in range(len(cumsum))], y=10*cumsum/np.linalg.norm(cumsum),
                     marker=dict(color='rgb(150, 25, 120)'))

layout = go.Layout(
    title="Cumulative Distribution Function"
)

fig = go.Figure(data=go.Data([trace1, trace2]), layout=layout)
iplot(fig, filename='cdf-dataset')

In [ ]:
ctrl_depr.columns